In [1]:
#multi_agent.ipynb
#
#by Joe Hahn
#jmh.datasciences@gmail.com
#12 February 2018
#
#This uses Q-learning on multiple agents to demonstrate something...

In [2]:
#game settings
N_agents = 10                #number of agents
N_buckets = 50               #number of buckets
max_turns = 200             #max number of moves in single game
turn = 0                    #current turn
strategy = 'random'         #agents move randomly
rn_seed = 14                #seed for random number generator

In [3]:
#import game
from multi_agent import *
import time
time_start = time.time()

Using TensorFlow backend.


In [4]:
#import plotting libraries
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.5, font='DejaVu Sans')

In [5]:
#initialize system
environment = initialize_environment(rn_seed, max_turns, N_buckets, N_agents)
print 'environment = ', environment
state = initialize_state(environment)
print 'state = ', state
reward, bucket_value = get_reward(state)
print 'reward = ', reward
print 'bucket_value = ', bucket_value
state_vector = state2vector(state, environment)
print 'state_vector = ', state_vector
game_state = get_game_state(turn, environment)
print 'game_state = ', game_state

environment =  {'max_turns': 200, 'rn_seed': 14, 'N_agents': 10, 'bucket_params': {'p0': array([ 0.        ,  0.00020408,  0.00040816,  0.00061224,  0.00081633,
        0.00102041,  0.00122449,  0.00142857,  0.00163265,  0.00183673,
        0.00204082,  0.0022449 ,  0.00244898,  0.00265306,  0.00285714,
        0.00306122, -0.00326531, -0.00346939,  0.00367347,  0.00387755,
        0.00408163,  0.00428571,  0.0044898 ,  0.00469388,  0.00489796,
        0.00510204,  0.00530612,  0.0055102 ,  0.00571429,  0.00591837,
        0.00612245,  0.00632653,  0.00653061,  0.00673469,  0.00693878,
        0.00714286,  0.00734694,  0.00755102,  0.0077551 ,  0.00795918,
        0.00816327,  0.00836735,  0.00857143,  0.00877551,  0.00897959,
        0.00918367, -0.01267347, -0.01294898, -0.01322449, -0.0135    ]), 'sigma': array([ 0.        ,  0.01428571,  0.02020305,  0.02474358,  0.02857143,
        0.03194383,  0.03499271,  0.03779645,  0.0404061 ,  0.04285714,
        0.0451754 ,  0.04738035,  0.

In [6]:
#illustrate moving an agent
if ((N_agents == 3) and (N_buckets == 10)):
    environment = initialize_environment(rn_seed, max_turns, N_buckets, N_agents)
    state = initialize_state(environment)
    print 'state = ', state
    print 'strategy = ', strategy
    state_moved, action = move_agents(state, environment, strategy)
    print 'state_moved = ', state_moved
    print 'action = ', action
    reward, bucket_value = get_reward(state_moved)
    print 'reward = ', reward
    print 'bucket_value = ', bucket_value
    state_next = update_state(state_moved, bucket_value, environment)
    print 'state_next = ', state_next
    state_vector_next = state2vector(state_next, environment)
    print 'state_vector_next = ', state_vector_next

In [7]:
#play 1 game of randomly-moving agents and stash history in dataframe
strategy = 'random'
memories = play_one_game(environment, strategy)
game_history, locations_rewards = memories2timeseries(memories, environment)
print 'number of memories generated during 1 game = ', len(memories)
game_history.tail()

number of memories generated during 1 game =  200


,turn,reward,agent_value_0,agent_value_1,agent_value_2,agent_value_3,agent_value_4,agent_value_5,agent_value_6,agent_value_7,agent_value_8,agent_value_9
195,195,16.900462,1.854802,4.595453,0.387646,2.695966,0.956435,1.311412,1.253760,1.321549,1.205139,1.398735
196,196,17.395461,1.883644,4.784250,0.393674,2.521678,0.967673,1.331805,1.147687,1.351256,1.195382,1.323412
197,197,17.077734,1.934566,5.127153,0.336644,2.623994,1.025785,1.349465,1.169622,1.387785,1.228690,1.211759
198,198,16.959896,1.853338,5.064076,0.347262,2.592600,1.035078,1.376552,1.038077,1.465710,1.127521,1.177521
199,199,16.586210,1.957566,4.984875,0.356880,2.627492,1.035376,1.353041,0.960421,1.355142,1.030216,1.298888


In [15]:
actions = locations_rewards.groupby('locations', as_index=False).agg({'reward':sum, 'N_visits':sum})
actions['reward_per_agent'] = actions['reward']*1.0/actions['N_visits']/N_agents
actions = actions.sort_values('reward_per_agent', ascending=False).reset_index(drop=True)
actions['action'] = actions.index
cols = ['action', 'locations', 'reward', 'N_visits', 'reward_per_agent']
actions = actions[cols]
actions.head(20)

,action,locations,reward,N_visits,reward_per_agent
0,0,"0, 1, 3, 6, 6, 9, 18, 23, 32, 44",17.905482,1,1.790548
1,1,"12, 19, 20, 21, 32, 33, 35, 35, 43, 49",17.884263,1,1.788426
2,2,"0, 12, 13, 20, 25, 27, 29, 45, 45, 49",17.881510,1,1.788151
3,3,"1, 6, 7, 13, 14, 17, 17, 27, 35, 41",17.842121,1,1.784212
4,4,"3, 3, 9, 16, 23, 27, 38, 44, 46, 47",17.785685,1,1.778569
5,5,"6, 9, 15, 16, 33, 34, 35, 40, 48, 49",17.783577,1,1.778358
6,6,"2, 11, 24, 28, 33, 34, 35, 40, 46, 46",17.612721,1,1.761272
7,7,"5, 12, 17, 18, 23, 23, 35, 41, 42, 43",17.557788,1,1.755779
8,8,"12, 12, 16, 16, 21, 24, 29, 32, 40, 44",17.532246,1,1.753225
9,9,"0, 5, 7, 10, 16, 18, 19, 20, 30, 47",17.527729,1,1.752773


In [ ]:
#m = memories[-1]
#state = m[1]
#action2locations = pd.DataFrame(state['action2locations'].copy())
#action2locations
#N = action2locations.groupby('locations_str', as_index=False).agg({'reward':sum, 'N_times':sum})
#N['reward_rate'] = N['reward']*1.0/N['N_times']
#N = N.sort_values('reward_rate', ascending=False).reset_index(drop=True)
#N['action'] = N.index
#cols = ['action', 'locations_str', 'reward', 'N_times', 'reward_rate']
#N = N[cols]
#N

In [ ]:
#plot relative reward versus turn, and agent values vs turn
df = game_history
xp = df['turn']
yp = df['reward']/df['reward'][0]
fig, ax = plt.subplots(1,1, figsize=(15, 8))
p = ax.set_title('relative reward vs turn')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('reward/reward[0]')
p = ax.plot(xp, yp)
#plot agents value versus turn
fig, ax = plt.subplots(1,1, figsize=(15, 8))
p = ax.set_title('agent value vs turn')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('agent value')
for col in df.columns:
    if ('agent_value_' in col):
        yp = df[col]
        p = ax.plot(xp, df[col], label=col)
p = ax.legend()

In [ ]:
#play a low-risk, low-reward game with all agents moving randomly among the lower third of buckets
strategy = 'low'
memories = play_one_game(environment, strategy)
game_history = memories2timeseries(memories, environment)
df = game_history
xp = df['turn']
yp = df['reward']/df['reward'][0]
fig, ax = plt.subplots(1,1, figsize=(15, 8))
p = ax.set_title('relative reward vs turn')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('reward/reward[0]')
p = ax.plot(xp, yp)
#plot agents value versus turn
fig, ax = plt.subplots(1,1, figsize=(15, 8))
p = ax.set_title('agent value vs turn')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('agent value')
for col in df.columns:
    if ('agent_value_' in col):
        yp = df[col]
        p = ax.plot(xp, df[col], label=col)
p = ax.legend()

In [ ]:
#play a medium-risk, medium-reward game with all agents moving randomly among the middle third of buckets
strategy = 'middle'
memories = play_one_game(environment, strategy)
game_history = memories2timeseries(memories, environment)
df = game_history
xp = df['turn']
yp = df['reward']/df['reward'][0]
fig, ax = plt.subplots(1,1, figsize=(15, 8))
p = ax.set_title('relative reward vs turn')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('reward/reward[0]')
p = ax.plot(xp, yp)
#plot agents value versus turn
fig, ax = plt.subplots(1,1, figsize=(15, 8))
p = ax.set_title('agent value vs turn')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('agent value')
for col in df.columns:
    if ('agent_value_' in col):
        yp = df[col]
        p = ax.plot(xp, df[col], label=col)
p = ax.legend()

In [ ]:
#play a high-risk, high-reward game with all agents moving randomly among the upper third of buckets
strategy = 'high'
memories = play_one_game(environment, strategy)
game_history = memories2timeseries(memories, environment)
df = game_history
xp = df['turn']
yp = df['reward']/df['reward'][0]
fig, ax = plt.subplots(1,1, figsize=(15, 8))
p = ax.set_title('relative reward vs turn')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('reward/reward[0]')
p = ax.plot(xp, yp)
#plot agents value versus turn
fig, ax = plt.subplots(1,1, figsize=(15, 8))
p = ax.set_title('agent value vs turn')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('agent value')
for col in df.columns:
    if ('agent_value_' in col):
        yp = df[col]
        p = ax.plot(xp, df[col], label=col)
p = ax.legend()

In [ ]:
#play 100 games using each strategy, and generate stats for each strategy
N_games = 100
strategies = ['low', 'middle', 'random', 'high']#, 'best']
game_histories = pd.DataFrame()
for strategy in strategies:
    print 'strategy = ', strategy 
    for game in range(N_games):
        turn = 0
        memories = play_one_game(environment, strategy)
        game_history = memories2timeseries(memories, environment)
        game_history['strategy'] = strategy
        game_history['game'] = game
        game_histories = game_histories.append(game_history)
game_stats = game_histories.groupby(['strategy', 'turn'])['reward'].agg(['mean', 'std'])
game_stats['std'] /= np.sqrt(N_games - 1)
game_stats.tail()

In [ ]:
#plot average agent value vs turn, for each strategy
fig, ax = plt.subplots(1,1, figsize=(15, 8))
p = ax.set_title('mean agent value vs strategy')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('mean agent value')
for strategy in strategies:
    reward = game_stats['mean'][strategy]
    std = game_stats['std'][strategy]
    std /= reward[0]
    reward /= reward[0]
    xp = reward.index
    yp = reward.values
    err = std.values
    p = ax.plot(xp, yp, label=strategy, markersize=4, marker='o')
    ax.errorbar(xp, yp, yerr=std, alpha=0.4, color=p[0].get_color())
p = ax.legend()

In [ ]:
stop-notebook

In [ ]:
#play 10 games making random actions, and stash moves in memories queue
N_games = 10
strategy = 'random'
memories = play_N_games(environment, strategy, N_games)
print 'number of memories = ', len(memories)

In [ ]:
#build queue of actions and rewards
N_actions = 10
actions_list = []
for memory in memories:
    turn, state, locations, state_next, reward, game_state = memory
    locations.sort()
    actions_list += [(locations, reward)]
actions = deque(maxlen=N_actions)
for j in range(N_actions):
    actions.append(random.choice(actions_list))
actions

In [ ]:
#build model
N_inputs = state_vector.shape[1]
N_outputs = N_actions
N_neurons = 100
model = build_model(N_inputs, N_neurons, N_outputs)
print model.summary()

In [ ]:
break notebook

In [ ]:
#train model
N_games = 20                               #number of games played during training
gamma = 0.85                                #discount for future rewards
batch_size = 100                             #number of memories used during experience-replay
debug = False                               #set True to see stats about each game's final turn
print 'batch_size = ', batch_size
print 'training model'
trained_model, game, rewards, epsilon = train(environment, model, N_games, gamma, memories, actions, batch_size, debug=debug)
print '\ntraining done'

In [ ]:
#plot rewards vs training game
fig, ax = plt.subplots(1,1, figsize=(15, 6))
xp = game
yp = rewards
p = ax.plot(xp, yp)
p = ax.set_title('reward vs training game')
p = ax.set_xlabel('game')
p = ax.set_ylabel('final reward')

In [ ]:
#plot epsilon vs game_number
fig, ax = plt.subplots(1,1, figsize=(15, 6))
xp = game
yp = epsilon
p = ax.plot(xp, yp)
p = ax.set_title('epsilon vs game number')
p = ax.set_xlabel('game number')
p = ax.set_ylabel('epsilon')

In [ ]:
#play one smart game
strategy = 'smart'
memories = play_one_game(environment, turn, strategy, model=trained_model)
game_history = memories2timeseries(memories, environment)
fig, ax = plt.subplots(1,1, figsize=(15, 6))
xp = game_history['turn']
yp = game_history['reward']
p = ax.plot(xp, yp)
p = ax.set_title('reward vs turn')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('reward')
game_history.head()

In [ ]:
#show grayscale of agent locations
cols = [col for col in game_history.columns if ('agents_' in col)]
df = game_history[cols]
fig, ax = plt.subplots(1,1, figsize=(15, 6))
plt.imshow(df.T, aspect='auto', cmap='gray')
df.tail(10)

In [ ]:
#plot bucket rewards versus turn
cols = ['turn'] + [col for col in game_history.columns if ('reward_' in col)]
df = game_history[cols].drop_duplicates()
cols = cols[1:]
fig, ax = plt.subplots(1,1, figsize=(15, 11))
p = ax.set_title('bucket rewards')
p = ax.set_xlabel('turn')
p = ax.set_ylabel('reward')
p = ax.set_xlim(0, df['turn'].max() + 40)
for col in cols:
    p = ax.plot(df['turn'], df[col], alpha=0.8, linewidth=1, label=col)
p = ax.legend(loc='lower right')

In [ ]:
#done!
time_stop = time.time()
print 'execution time (minutes) = ', (time_stop - time_start)/60.0

In [ ]:
pd.DataFrame.append?

In [ ]:
loc = np.array([0, 2, 5])
loc

In [ ]:
str(loc.tolist())

In [ ]:
s='[1,2,3]'
s.split(',')